In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from sklearn import preprocessing
import torch

import gb

import tsvar

---

## 1. Simulation 

Define model parameters

In [3]:
mu = torch.tensor([0.1, 0.1, 0.1, 0.1])

beta = torch.tensor([1.1, 1.1, 1.1, 1.1])

alpha = torch.tensor([
    [0.7, 0.7, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.0],
    [0.0, 0.0, 0.0, 0.7],
    [0.0, 0.7, 0.7, 0.0]
])


dim = 4
n_params = len(mu) + len(beta) + len(alpha.flatten())

coeffs_true = torch.cat((mu, beta, alpha.flatten()))

Simulate some data

In [4]:
end_time = 500000.0

wold_sim = tsvar.simulate.GrangeBuscaSimulator(mu_rates=mu, Alpha_ba=alpha, Beta_b=beta)
events = wold_sim.simulate(end_time)
events_t = [torch.tensor(ev, dtype=torch.float) for ev in events]

print(f"Simulated {sum(map(len, events_t))} events with end time: {end_time}")
print(list(map(len, events_t)))

AttributeError: module 'tsvar.simulate' has no attribute 'GrangeBuscaSimulator'

---

## 2. Inference

### 2.1. GrangerBusca

Run the GrangerBusca algorithm based on Gibbs sampling.

In [88]:
granger_model = gb.GrangerBusca(
    alpha_prior=0.1, num_iter=2000,
    metropolis=False, sloppy=1, beta_strategy=1.0)
granger_model.fit(events);

print()
print('Mu:')
print(granger_model.mu_.round(2))
print('Beta:')
print(granger_model.beta_.round(2))
print('Alpha:')
print(sklearn.preprocessing.normalize(granger_model.Alpha_.toarray()).round(2))

{0: 3165, 1: 2828, 2: 1970, 3: 1447}
{0: array([2], dtype=uint64), 1: array([1], dtype=uint64), 2: array([3], dtype=uint64), 3: array([0], dtype=uint64)}

Mu:
[0.12 0.24 0.27 0.17]
Beta:
[1. 1. 1. 1.]
Alpha:
[[0.64 0.75 0.   0.15]
 [0.23 0.54 0.8  0.13]
 [0.18 0.01 0.55 0.81]
 [0.   0.71 0.69 0.15]]


In [47]:
alpha.numpy().round(2)

array([[0.7, 0.7, 0. , 0. ],
       [0. , 0. , 0.7, 0. ],
       [0. , 0. , 0. , 0.7],
       [0. , 0.7, 0.7, 0. ]], dtype=float32)

### 2.2. Run BBVI

In [64]:
x0 = torch.cat((
    2.0 * torch.ones(dim, dtype=torch.float),
    3.0 * torch.ones(dim, dtype=torch.float),
    0.5 * torch.ones((dim, dim), dtype=torch.float).flatten()
))
 
optimizer = torch.optim.Adam([x0], lr=0.03)

learner_mle = tsvar.learners.MLELearner(
    model=tsvar.wold_model.WoldModel(), 
    prior=tsvar.priors.GaussianLaplacianPrior(
        C=torch.ones(n_params, dtype=torch.float) * 100.0,
        mask_gaus=torch.cat((
            torch.ones(2 * dim, dtype=torch.bool),
            torch.zeros(dim * dim, dtype=torch.bool)
        ))
    ),
    optimizer=optimizer, tol=1e-5, max_iter=10000, debug=False)

learner_callback_mle = tsvar.utils.callbacks.LearnerCallbackMLE(x0, coeffs_true.numpy(), print_every=100)

coeffs_hat_mle = learner_mle.fit(events_t, end_time, x0, callback=learner_callback_mle).detach().numpy()

iter: 10000 | dx: 3.5862e-03 | loss: 1.5441e+04    

In [80]:
print('MLE estimation:')
print('mu:')
print(coeffs_hat_mle[:dim].round(2))
print('beta:')
print(1 + coeffs_hat_mle[dim:2*dim].round(2))
print('alpha:')
print(np.reshape(coeffs_hat_mle[2*dim:], (dim, dim)).round(2))
print()
print('Ground truth:')
print('mu:')
print(coeffs_true[:dim].numpy().round(2))
print('beta:')
print(coeffs_true[dim:2*dim].numpy().round(2))
print('alpha:')
print(np.reshape(coeffs_true[2*dim:], (dim, dim)).numpy().round(2))

MLE estimation:
mu:
[0.08 0.09 0.06 0.11]
beta:
[1.24 1.12 1.01 1.06]
alpha:
[[0.77 0.8  0.02 0.03]
 [0.   0.01 0.71 0.01]
 [0.01 0.02 0.05 0.67]
 [0.06 0.67 0.68 0.  ]]

Ground truth:
mu:
[0.1 0.1 0.1 0.1]
beta:
[1.1 1.1 1.1 1.1]
alpha:
[[0.7 0.7 0.  0. ]
 [0.  0.  0.7 0. ]
 [0.  0.  0.  0.7]
 [0.  0.7 0.7 0. ]]


---

In [77]:
from tsvar.wold_model import WoldModel
from tsvar.models import ModelVariational
from tsvar.priors import GaussianLaplacianPrior
from tsvar.posteriors import LogNormalPosterior
from tsvar.learners import VariationalInferenceLearner

# model_wold = WoldModelSimple()
# model_wold.set_beta(beta)
# n_params = len(mu) + len(A.flatten())

model_wold = WoldModel()
n_params = len(mu) + len(beta) + len(alpha.flatten())

model_var = ModelVariational(
    model=model_wold,
    prior=GaussianLaplacianPrior(
        C=torch.ones(n_params, dtype=torch.float) * 10.0,
        mask_gaus=torch.cat((
            torch.ones(dim, dtype=torch.bool),
            torch.zeros(dim + dim * dim, dtype=torch.bool)
        ))
    ),
    posterior=LogNormalPosterior(),
    n_samples=1,
)

coeffs_true = torch.cat((mu, beta, alpha.flatten()))

x0 = torch.tensor(np.hstack((
    np.random.normal(loc=0.1, scale=0.1, size=dim),
    np.random.normal(loc=0.1, scale=0.1, size=dim),
    np.random.normal(loc=0.1, scale=0.1, size=dim*dim),
    
    np.log(np.random.normal(loc=1e-5, scale=0.000001, size=dim)),
    np.log(np.random.normal(loc=1e-5, scale=0.000001, size=dim)),
    np.log(np.random.normal(loc=1e-5, scale=0.000001, size=dim*dim)),
)), dtype=torch.float)

print('--------- x0:')
print('alpha0:', x0[:n_params])
print('beta0:', x0[n_params:])
print()

optimizer = torch.optim.Adagrad([x0])

learner_callback = tsvar.utils.callbacks.LearnerCallbackMLE(x0, coeffs_true.numpy(), print_every=100)

learner = VariationalInferenceLearner(
    model=model_var, optimizer=optimizer, lr=0.1,
    tol=1e-6, 
    lr_gamma=0.9999, max_iter=5000,
    hyperparam_interval=1000, hyperparam_offset=np.inf, hyperparam_momentum=0.0,
)

coeffs_hat_vi = learner.fit(events_t, end_time, x0, callback=learner_callback)

--------- x0:
alpha0: tensor([ 0.1150,  0.0137,  0.1406,  0.1430,  0.1756,  0.0426,  0.0099, -0.1360,
         0.1301,  0.1368,  0.2858,  0.0255, -0.1056,  0.0451,  0.1138,  0.0924,
         0.0922,  0.0919,  0.1142,  0.1587,  0.2003,  0.0498,  0.2511,  0.3000])
beta0: tensor([-11.4825, -11.4916, -11.4771, -11.6889, -11.6135, -11.6036, -11.4957,
        -11.5414, -11.3675, -11.3810, -11.6354, -11.6472, -11.4665, -11.4827,
        -11.4458, -11.3644, -11.4595, -11.3844, -11.5981, -11.4948, -11.6035,
        -11.4157, -11.8216, -11.6248])

iter:  5000 | dx: 5.8867e-03 | loss: 1.5676e+04    

In [81]:
coeffs_hat_vi_mode = learner.model.posterior.mode(coeffs_hat[:n_params], coeffs_hat[n_params:]).detach().numpy().round(2)
coeffs_hat_vi_std = np.sqrt(learner.model.posterior.variance(coeffs_hat[:n_params], coeffs_hat[n_params:]).detach().numpy().round(2))


print('==== VI estimation:')
print('--- mu:')
print('mode:')
print(coeffs_hat_vi_mode[:dim].round(2))
print('std:')
print(coeffs_hat_vi_std[:dim].round(2))
print('--- beta:')
print('mode:')
print(1 + coeffs_hat_vi_mode[dim:2*dim].round(2))
print('std:')
print(coeffs_hat_vi_std[dim:2*dim].round(2))
print('--- alpha:')
print('mode:')
print(np.reshape(coeffs_hat_vi_mode[2*dim:], (dim, dim)).round(2))
print('std:')
print(np.reshape(coeffs_hat_vi_std[2*dim:], (dim, dim)).round(2))
print()
print('==== Ground truth:')
print('mu:')
print(coeffs_true[:dim].numpy().round(2))
print('beta:')
print(coeffs_true[dim:2*dim].numpy().round(2))
print('alpha:')
print(np.reshape(coeffs_true[2*dim:], (dim, dim)).numpy().round(2))

==== VI estimation:
--- mu:
mode:
[0.11 0.18 0.28 0.18]
std:
[0. 0. 0. 0.]
--- beta:
mode:
[3.05 5.27 4.89 3.2 ]
std:
[0.66 2.16 1.22 0.55]
--- alpha:
mode:
[[0.53 0.67 0.34 0.22]
 [0.17 0.34 0.5  0.22]
 [0.17 0.29 0.36 0.4 ]
 [0.17 0.89 0.8  0.31]]
std:
[[0.17 0.2  0.1  0.  ]
 [0.   0.1  0.14 0.1 ]
 [0.   0.1  0.1  0.1 ]
 [0.   0.14 0.2  0.1 ]]

==== Ground truth:
mu:
[0.1 0.1 0.1 0.1]
beta:
[1.1 1.1 1.1 1.1]
alpha:
[[0.7 0.7 0.  0. ]
 [0.  0.  0.7 0. ]
 [0.  0.  0.  0.7]
 [0.  0.7 0.7 0. ]]
